In [31]:
import tensorflow as tf
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import numpy as np
import pickle

## Reading spambase dataset

In [2]:
traininig_data = pd.read_csv('./spambase/spambase.data')

In [3]:
traininig_data.fillna(0,inplace=True)

In [4]:
traininig_data.shape

(4600, 58)

In [5]:
X = traininig_data[traininig_data.columns[0:56]].values
y = traininig_data[traininig_data.columns[57]].values

In [7]:
def one_hot_encoding(x):
    n = len(x)
    n_unique = len(np.unique(x))
    one_hot_encode = np.zeros([n,n_unique])
    one_hot_encode[np.arange(n),x] = 1
    return one_hot_encode

In [8]:
Y = one_hot_encoding(y)

## Read Gaussian Dataset

In [32]:
X = pickle.load(open('./gaussian_data_separable/normal_dataset_X.pkl','rb'))

In [33]:
y = pickle.load(open('./gaussian_data_separable/normal_dataset_Y.pkl','rb'))

In [41]:
y = np.asarray(y)
y_new = []
for i in range(y.shape[0]):
    if y[i] == -1:
        y_new.append(0)
    else:
        y_new.append(1)

In [42]:
y = np.asarray(y_new)

In [43]:
Y = one_hot_encoding(y)

In [66]:
train_X = X[0:3000,:]
train_Y = Y[0:3000,:]
test_X = X[3000:3999,:]
test_Y = Y[3000:3999,:]

In [10]:
# train_X,test_X,train_Y,test_Y = train_test_split(X,Y,test_size=0.3,random_state=42)

In [67]:
train_X.shape

(3000, 2)

In [68]:
# define parameters
learning_rate= 0.1
dimension = X.shape[1]
epoch = 1000

# define number of neuron for each hidden layer
n_layer_1 = dimension
n_layer_2 = dimension
n_layer_3 = dimension
n_layer_4 = dimension
n_classes = len(np.unique(y))

In [69]:
x = tf.placeholder(tf.float32,[None,dimension],name='x')
_y = tf.placeholder(tf.float32,[None,n_classes],name='y')

In [70]:
def multilayer_perceptron(X,weights,biases):
    
    layer_1 = tf.add(tf.matmul(X,weights['h1']),biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3 = tf.nn.sigmoid(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4 = tf.nn.relu(layer_4)
    
    out_layer = tf.add(tf.matmul(layer_4,weights['out']),biases['out'])
    return out_layer

In [71]:
weights = {
    'h1':tf.Variable(tf.truncated_normal([dimension,n_layer_1]), name = 'h1'),
    'h2':tf.Variable(tf.truncated_normal([n_layer_1,n_layer_2]), name = 'h2'),
    'h3':tf.Variable(tf.truncated_normal([n_layer_2,n_layer_3]), name = 'h3'),
    'h4':tf.Variable(tf.truncated_normal([n_layer_3,n_layer_4]), name = 'h4'),
    'out':tf.Variable(tf.truncated_normal([n_layer_4,n_classes]), name = 'h_out')
}

biases = {
    'b1':tf.Variable(tf.truncated_normal([n_layer_1]), name = 'b1'),
    'b2':tf.Variable(tf.truncated_normal([n_layer_2]), name = 'b2'),
    'b3':tf.Variable(tf.truncated_normal([n_layer_3]), name = 'b3'),
    'b4':tf.Variable(tf.truncated_normal([n_layer_4]), name = 'b4'),
    'out':tf.Variable(tf.truncated_normal([n_classes]), name = 'b_out')
}

In [72]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [73]:
saver = tf.train.Saver()
tf.summary.FileWriter('./graph',sess.graph)

In [74]:
y_output = multilayer_perceptron(x,weights,biases)

In [75]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_output,labels=_y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
training_step = optimizer.minimize(loss_function)

In [77]:
best_accuracy = 0
for i in range(0,epoch):
    sess.run(training_step,feed_dict={x:train_X,_y:train_Y})
    cost = sess.run(loss_function,feed_dict={x:train_X,_y:train_Y})
    accuracy = sess.run(tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y_output,1),tf.argmax(_y,1)),tf.float32)),feed_dict={x:train_X,_y:train_Y})
    
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        print('epoch: ',i,' , cost: ',cost,' accuracy: ',accuracy)
    


epoch:  0  , cost:  0.67337775  accuracy:  0.78033334
epoch:  1  , cost:  0.67323273  accuracy:  0.7816667
epoch:  2  , cost:  0.67308605  accuracy:  0.783
epoch:  4  , cost:  0.6727887  accuracy:  0.7836667
epoch:  5  , cost:  0.6726375  accuracy:  0.784
epoch:  7  , cost:  0.67233074  accuracy:  0.78433335
epoch:  12  , cost:  0.6715357  accuracy:  0.78466666
epoch:  13  , cost:  0.67137164  accuracy:  0.78533334
epoch:  14  , cost:  0.6712058  accuracy:  0.7866667
epoch:  15  , cost:  0.6710384  accuracy:  0.7876667
epoch:  19  , cost:  0.6703507  accuracy:  0.78866667
epoch:  20  , cost:  0.67017406  accuracy:  0.78933334
epoch:  21  , cost:  0.66999567  accuracy:  0.79033333
epoch:  22  , cost:  0.66981536  accuracy:  0.79066664
epoch:  23  , cost:  0.66963303  accuracy:  0.791
epoch:  25  , cost:  0.6692625  accuracy:  0.7913333
epoch:  26  , cost:  0.66907436  accuracy:  0.7926667
epoch:  27  , cost:  0.668884  accuracy:  0.793
epoch:  28  , cost:  0.66869175  accuracy:  0.79433

epoch:  212  , cost:  0.53198946  accuracy:  0.89033335
epoch:  213  , cost:  0.5301277  accuracy:  0.891
epoch:  214  , cost:  0.5282484  accuracy:  0.89133334
epoch:  215  , cost:  0.52635115  accuracy:  0.892
epoch:  216  , cost:  0.524437  accuracy:  0.893
epoch:  217  , cost:  0.5225064  accuracy:  0.894
epoch:  218  , cost:  0.52055854  accuracy:  0.89433336
epoch:  219  , cost:  0.5185938  accuracy:  0.895
epoch:  220  , cost:  0.51661146  accuracy:  0.89533335
epoch:  221  , cost:  0.5146134  accuracy:  0.897
epoch:  222  , cost:  0.5125987  accuracy:  0.8973333
epoch:  223  , cost:  0.5105667  accuracy:  0.8983333
epoch:  224  , cost:  0.5085187  accuracy:  0.89966667
epoch:  225  , cost:  0.5064534  accuracy:  0.9
epoch:  227  , cost:  0.5022761  accuracy:  0.90033334
epoch:  228  , cost:  0.500164  accuracy:  0.90133333
epoch:  229  , cost:  0.49803653  accuracy:  0.90166664
epoch:  230  , cost:  0.49589518  accuracy:  0.9023333
epoch:  231  , cost:  0.49373862  accuracy:  0

epoch:  435  , cost:  0.14212942  accuracy:  0.99666667
epoch:  462  , cost:  0.12573716  accuracy:  0.997
epoch:  495  , cost:  0.11006835  accuracy:  0.99733335
epoch:  801  , cost:  0.051761013  accuracy:  0.99766666


In [19]:
saver.save(sess,'./model_1')
sess.close()

## Prediction

In [ ]:
train_X.shape

In [ ]:
# graph = tf.Graph()
# with graph.as_default():
#     sess = tf.Session()
#     saver = tf.train.import_meta_graph("{}.meta".format('./model_1'))
#     model = saver.restore(sess,'./model_1')
# #     h1 = graph.get_operation_by_name('layer_1')
# #     h1_val = sess.run(h1,feed_dict={X:test_X[0].reshape(19,1)})
# #     print(h1_val)

In [20]:
sess = tf.Session()
saver = tf.train.import_meta_graph("{}.meta".format('./model_1'))
model = saver.restore(sess,'./model_1')

INFO:tensorflow:Restoring parameters from ./model_1


In [21]:
predicted_probabilities = tf.nn.softmax(y_output)
predicted_output = tf.argmax(y_output,1)
confidence_score = tf.reduce_max(y_output)
# correct_prediction = tf.equal(prediction,tf.argmax(_y,1))
# accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [25]:
predicted_probabilities = tf.nn.softmax(y_output)
predicted_output = tf.argmax(y_output,1)
confidence_score = tf.reduce_max(y_output)

In [22]:
accuracy_list = []
for i in range(0,len(test_X)):
    prediction_probability = sess.run(predicted_probabilities,feed_dict={x:test_X[i].reshape(1,19)})
    prediction_output = sess.run([predicted_output,confidence_score],feed_dict={x:test_X[i].reshape(1,19)})
#     accuracy_output = sess.run(accuracy,feed_dict={x:test_X[i].reshape(1,19),_y:test_Y[i].reshape(1,4)})
#     accuracy_list.append(accuracy_output)
#     print('original: ',np.argmax(test_Y[i]),' , predicted: ',prediction_output,' , accuracy: ',accuracy_output)
    print(prediction_output ,' : ',prediction_probability)

[array([1]), -0.6990948]  :  [[0.18575607 0.47173178 0.15223877 0.19027342]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([1]), -0.81722736]  :  [[0.19295168 0.45185742 0.34383523 0.0113557 ]]
[array([3]), -0.6685483]  :  [[0.10249227 0.08057756 0.15120429 0.6657259 ]]
[array([3]), -2.555659]  :  [[0.2603322  0.08558729 0.3067105  0.34737003]]
[array([2]), -2.5378003]  :  [[0.3113574  0.13880344 0.36941642 0.18042278]]
[array([3]), -1.4995766]  :  [[0.16297552 0.19121425 0.22934908 0.4164611 ]]
[array([3]), -2.5438015]  :  [[0.28203636 0.15634242 0.2764466  0.28517458]]
[array([2]), -2.6447992]  :  [[0.26581568 0.08531164 0.36103383 0.28783885]]
[array([2]), -2.0701656]  :  [[0.28619507 0.13940364 0.3409439  0.23345745]]
[array([3]), -2.1426978]  :  [[0.24709263 0.11935169 0.26460296 0.3689527 ]]
[array([1]), -1.6099167]  :  [[0.3004404  0.4441442  0.14700869 0.10840679]]
[array([2]), -2.2381568]  :  [[0.24306273 0.074116   0.4024231  0.28039813]]

[array([3]), -1.7850599]  :  [[0.1986381  0.09886689 0.30581594 0.3966791 ]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([1]), -1.7114816]  :  [[0.2661246  0.3225207  0.21929602 0.1920587 ]]
[array([2]), -2.0701656]  :  [[0.28619507 0.13940364 0.3409439  0.23345745]]
[array([2]), -2.0701656]  :  [[0.28619507 0.13940364 0.3409439  0.23345745]]
[array([2]), -2.5378003]  :  [[0.3113574  0.13880344 0.36941642 0.18042278]]
[array([3]), -0.6685483]  :  [[0.10249227 0.08057756 0.15120429 0.6657259 ]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([1]), -1.7114816]  :  [[0.2661246  0.3225207  0.21929602 0.1920587 ]]
[array([3]), -1.7850599]  :  [[0.1986381  0.09886689 0.30581594 0.3966791 ]]
[array([1]), -1.7114816]  :  [[0.2661246  0.3225207  0.21929602 0.1920587 ]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]

[array([1]), -1.22082]  :  [[0.2622825  0.46736374 0.22986732 0.04048648]]
[array([3]), -2.555659]  :  [[0.2603322  0.08558729 0.3067105  0.34737003]]
[array([1]), -1.22082]  :  [[0.2622825  0.46736374 0.22986732 0.04048648]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([1]), -0.936522]  :  [[0.21806115 0.42532566 0.3301062  0.02650698]]
[array([1]), -1.1998749]  :  [[0.2824204  0.49079958 0.20044129 0.02633878]]
[array([1]), -1.7114816]  :  [[0.2661246  0.3225207  0.21929602 0.1920587 ]]
[array([3]), -1.7850599]  :  [[0.1986381  0.09886689 0.30581594 0.3966791 ]]
[array([1]), -1.1998749]  :  [[0.2824204  0.49079958 0.20044129 0.02633878]]
[array([1]), -0.12212467]  :  [[0.15116294 0.540098   0.24906746 0.05967158]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([1]), -1.7114816]  :  [[0.2661246  0.3225207  0.21929602 0.1920587 ]]
[array([0]), -2.3816965]  :  [[0.32047066 0.1916533  0.2283173  0.2595587 ]]
[arr

[array([1]), -1.22082]  :  [[0.2622825  0.46736374 0.22986732 0.04048648]]
[array([2]), -1.9877732]  :  [[0.24796434 0.26453263 0.2909972  0.19650583]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([1]), -1.7114816]  :  [[0.2661246  0.3225207  0.21929602 0.1920587 ]]
[array([1]), -1.7085781]  :  [[0.2659818  0.35560954 0.14566404 0.23274456]]
[array([1]), -1.22082]  :  [[0.2622825  0.46736374 0.22986732 0.04048648]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([2]), -2.5378003]  :  [[0.3113574  0.13880344 0.36941642 0.18042278]]
[array([3]), -0.6685483]  :  [[0.10249227 0.08057756 0.15120429 0.6657259 ]]
[array([3]), -2.5438015]  :  [[0.28203636 0.15634242 0.2764466  0.28517458]]
[array([1]), -1.6170473]  :  [[0.30003035 0.4441288  0.14589958 0.10994123]]
[array([3]), -1.5217955]  :  [[0.18864517 0.21466891 0.20061171 0.3960742 ]]
[array([0]), -2.3816965]  :  [[0.32047066 0.1916533  0.2283173  0.2595587 ]]
[ar